In [ ]:
import os
import time
import pandas as pd
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm.auto import tqdm

file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}


def button_click(driver, path):
    """원하는 부분을 직접 클릭하는 함수"""
    button = driver.find_element(By.XPATH, path)
    driver.execute_script("arguments[0].click();", button)

url = "https://www.medicalkorea.or.kr/en/registeredhospitals"

In [ ]:
# 접속
driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(5)

title_result = []
url_result = []
address_result = []
etc_result = []

for i in tqdm(range(1,367)):
    for j in range(1,7):
        title = driver.find_element(By.XPATH, f'//*[@id="container"]/div[1]/div/div[2]/ul/li[{j}]/div[1]')
        try:
            title_result.append(title.text)
        except:
            title_result.append("")

        # 활성화버튼 누르기
        button_click(driver, f'//*[@id="container"]/div[1]/div/div[2]/ul/li[{j}]/div[1]/a')

        try:
            etc_text = driver.find_element(By.XPATH, f'//*[@id="container"]/div[1]/div/div[2]/ul/li[{j}]/div[2]/div/ul[3]')
            etc_result.append(etc_text.text)

            try:
                address_text = driver.find_element(By.XPATH, f'//*[@id="container"]/div[1]/div/div[2]/ul/li[{j}]/div[2]/div/ul[2]')
                address_result.append(address_text.text)
            except:
                address_result.append("")

            try:
                url_text = driver.find_element(By.XPATH, f'//*[@id="container"]/div[1]/div/div[2]/ul/li[{j}]/div[2]/div/ul[1]')
                url_result.append(url_text.text)
            except:
                url_result.append("")

        except:
            url_result.append("")

            try:
                address_text = driver.find_element(By.XPATH, f'//*[@id="container"]/div[1]/div/div[2]/ul/li[{j}]/div[2]/div/ul[1]')
                address_result.append(address_text.text)
            except:
                address_result.append("")

            try:
                etc_text = driver.find_element(By.XPATH, f'//*[@id="container"]/div[1]/div/div[2]/ul/li[{j}]/div[2]/div/ul[2]')
                etc_result.append(etc_text.text)
            except:
                etc_result.append("")

    # 다음버튼
    time.sleep(2)
    
    if i == 0:
        button_click(driver, '//*[@id="container"]/div[1]/div/div[3]/a[5]')
    elif i == 1:
        button_click(driver, '//*[@id="container"]/div[1]/div/div[3]/a[6]')
    elif i > 1 and i <= 364:
        button_click(driver, '//*[@id="container"]/div[1]/div/div[3]/a[10]')
    elif i == 365:
        button_click(driver, '//*[@id="container"]/div[1]/div/div[3]/a[9]')
    else:
        pass
    
driver.quit()

In [ ]:
df_list = [title_result, address_result, url_result, etc_result]

df = pd.DataFrame(df_list).transpose()
df.rename(columns={0:"title", 1:"address", 2:"url", 3:"etc"}, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.drop_duplicates(subset=["title"])

In [ ]:
df["CODE"] = df["title"].apply(lambda x:x.split("\n")[0])
df["NAME"] = df["title"].apply(lambda x:x.split("\n")[1])
df["AREA1"] = df["title"].apply(lambda x:x.split("\n")[2])

def tel_check(input_text):
    for i in range(len(input_text.split("\n"))):
        if "Tel" in input_text.split("\n")[i]:
            return input_text.split("\n")[i].replace("Tel","")
    return ""

def fax_check(input_text):
    for i in range(len(input_text.split("\n"))):
        if "Fax" in input_text.split("\n")[i]:
            return input_text.split("\n")[i].replace("Fax","")
    return ""

def mail_check(input_text):
    for i in range(len(input_text.split("\n"))):
        if "E-Mail" in input_text.split("\n")[i]:
            return input_text.split("\n")[i].replace("E-Mail","")
    return ""

df["TEL"] = df["etc"].apply(lambda x:tel_check(x))
df["FAX"] = df["etc"].apply(lambda x:fax_check(x))
df["E-MAIL"] = df["etc"].apply(lambda x:mail_check(x))

df = df[["CODE", "NAME", "AREA1", "address", "url", "TEL", "FAX", "E-MAIL"]]

In [ ]:
df.to_excel(f"{file_path}의료 데이터 확인.xlsx", index=False)

# 한글번역

In [ ]:
import re
import pymysql

tqdm.pandas()

def dev_db_conn(): 
    """개발 DB 접속 함수""" 
    conn = pymysql.connect( 
        host = 'db-7ma06.pub-cdb.ntruss.com', 
        user = 'redtable', 
        password = 'fpemxpdlqmf5491!@#', 
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor) 
    return conn

def insert_comma(s):
    s1 = re.sub(r'(\d)(\s+)([가-힣A-Za-z])', r'\1, \3', s)
    return re.sub(r'([가-힣A-Za-z])(\s+)([가-힣A-Za-z])', r'\1, \3', s1)

with dev_db_conn() as conn:
    sql = """
    SELECT T1.SHIDO_NM, T1.EN
    FROM dictionary.shido T1
    """
    shido = pd.read_sql(sql, conn)
    
    sql = """
    SELECT T1.SHIDO_NM, T1.SHIGUNGU_NM, T1.EN
    FROM dictionary.shigungu T1
    """
    shigungu = pd.read_sql(sql, conn)
    
    sql = """
    SELECT T1.SHIDO_NM, T1.SHIGUNGU_NM, T1.EUPMYEONDONG_NM, T1.EN
    FROM dictionary.eupmyeondong T1
    """
    eupmyeondong = pd.read_sql(sql, conn)
    
    sql = """
    SELECT T1.SHIDO_NM, T1.SHIGUNGU_NM, T1.SHIGUNGU_NM, T1.DORO_NM, T1.EN
    FROM dictionary.doro T1
    GROUP BY T1.SHIGUNGU_NM, T1.DORO_NM
    """
    doro = pd.read_sql(sql, conn)
    
def en_to_ko(input_text):
    input_text = input_text.replace(", South Korea", "")
    input_text = re.sub('\([^)]+\)', '', input_text).replace(" ,", ",")
    input_text = insert_comma(input_text)
    input_list = input_text.split(", ")

    number_result = []
    doro_result = []
    eupmyeondong_result = []
    shigungu_result = []
    shido_result = []

    for i in input_list:
        if i.upper() == i.lower():
            number_result.append(i)
            continue

        if i in shido["EN"].tolist():
            with dev_db_conn() as conn:
                sql = f"""
                SELECT DISTINCT T1.SHIDO_NM
                FROM dictionary.shido T1
                WHERE T1.EN = "{i}";
                """
                match_shido = pd.read_sql(sql, conn)
                shido_result.append(match_shido["SHIDO_NM"][0])

        if i in shigungu["EN"].tolist():
            with dev_db_conn() as conn:
                sql = f"""
                SELECT DISTINCT T1.SHIGUNGU_NM
                FROM dictionary.shigungu T1
                WHERE T1.EN = "{i}";
                """
                match_shigungu = pd.read_sql(sql, conn)
                shigungu_result.append(match_shigungu["SHIGUNGU_NM"][0])

        if i in eupmyeondong["EN"].tolist():
            with dev_db_conn() as conn:
                sql = f"""
                SELECT DISTINCT T1.EUPMYEONDONG_NM
                FROM dictionary.eupmyeondong T1
                WHERE T1.EN = "{i}";
                """
                match_eupmyeondong = pd.read_sql(sql, conn)
                eupmyeondong_result.append(match_eupmyeondong["EUPMYEONDONG_NM"][0])

        if i in doro["EN"].tolist():
            with dev_db_conn() as conn:
                sql = f"""
                SELECT DISTINCT T1.DORO_NM
                FROM dictionary.doro T1
                WHERE T1.EN = "{i}";
                """
                match_doro = pd.read_sql(sql, conn)
                doro_result.append(match_doro["DORO_NM"][0])

    final_address = ' '.join(shido_result) + " " +' '.join(shigungu_result) + " " + ' '.join(eupmyeondong_result) + " " + ' '.join(doro_result) + " " + ' '.join(number_result)
    final_address = final_address.replace("  ", " ")
    return final_address

In [ ]:
df["ko_address"] = df["address"].progress_apply(lambda x:en_to_ko(x))